In [11]:
import numpy as np
import mindspore
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Parameter, Tensor, ms_function

In [12]:
class TextCNN(nn.Cell):
    def __init__(self, embedding_size, sequence_length, num_classes, filter_sizes, num_filters, vocab_size):
        super(TextCNN, self).__init__()
        self.num_filters_total = num_filters * len(filter_sizes)
        self.filter_sizes = filter_sizes
        self.sequence_length = sequence_length
        self.W = nn.Embedding(vocab_size, embedding_size)
        self.Weight = nn.Dense(self.num_filters_total, num_classes, has_bias=False)
        self.Bias = Parameter(Tensor(np.ones(num_classes), mindspore.float32), name='bias')
        self.filter_list = nn.CellList()
        for size in filter_sizes:
            seq_cell = nn.SequentialCell([
                nn.Conv2d(1, num_filters, (size, embedding_size), pad_mode='valid'),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=(sequence_length - size + 1, 1))
            ])
            self.filter_list.append(seq_cell)

    def construct(self, X):
        embedded_chars = self.W(X)
        embedded_chars = embedded_chars.expand_dims(1)
        pooled_outputs = []
        for conv in self.filter_list:
            pooled = conv(embedded_chars)
            pooled = pooled.transpose((0, 3, 2, 1))
            pooled_outputs.append(pooled)
            
        h_pool = ops.concat(pooled_outputs, len(self.filter_sizes))
        h_pool_flat = h_pool.view(-1, self.num_filters_total)
        model = self.Weight(h_pool_flat) + self.Bias
        return model

In [13]:

embedding_size = 2
sequence_length = 3
num_classes = 2
filter_sizes = [2, 2, 2]
num_filters = 3

sentences = ["i love you", "he loves me", "she likes baseball", " i hate you", "sorry for that", "this is awful"]
labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good.

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
vocab_size = len(word_dict)

model = TextCNN(embedding_size, sequence_length, num_classes, filter_sizes, num_filters, vocab_size)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = nn.Adam(model.trainable_params(), learning_rate=0.001)

In [15]:
inputs = Tensor([np.asarray([word_dict[n] for n in sen.split()]) for sen in sentences], mindspore.int32)
targets = Tensor([out for out in labels], mindspore.int32) 

In [16]:
def forward(inputs, targets):
    logits = model(inputs)
    loss = criterion(logits, targets)
    return loss

In [17]:
grad_fn = ops.value_and_grad(forward, None, optimizer.parameters)

In [18]:
@ms_function
def train_step(inputs, targets):
    loss, grads = grad_fn(inputs, targets)
    optimizer(grads)
    return loss

In [19]:
model.set_train()

epoch = 5000
for step in range(epoch):
    loss = train_step(inputs, targets)
    
    if (step + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (step + 1), 'cost =', '{:.6f}'.format(loss.asnumpy()))

Epoch: 1000 cost = 0.002617
Epoch: 2000 cost = 0.000449
Epoch: 3000 cost = 0.000152
Epoch: 4000 cost = 0.000066
Epoch: 5000 cost = 0.000031


In [20]:
test_text = 'sorry hate you'
tests = [np.asarray([word_dict[n] for n in test_text.split()])]
test_batch = Tensor(tests, mindspore.int32)

# Predict
predict = model(test_batch).asnumpy().argmax(1)
if predict[0] == 0:
    print(test_text,"is Bad Mean...")
else:
    print(test_text,"is Good Mean!!")

sorry hate you is Bad Mean...
